# **Using the Swarmauri Package Components**

This guide demonstrates how to use various components of the Swarmauri package, including the `MistralModel`, `Conversation`, and message classes such as `HumanMessage`, `SystemMessage`. These examples will guide you through the setup and utilization of these components for common tasks.

## **Setup**

Before using the components, ensure you have the necessary API keys set up as environment variables. In this case, you'll need the `MISTRAL_API_KEY` for initializing the `MistralModel`.

In [10]:
import pytest
import os
from swarmauri.standard.llms.concrete.MistralModel import MistralModel as LLM
from swarmauri.standard.conversations.concrete.Conversation import Conversation

from swarmauri.standard.messages.concrete.AgentMessage import AgentMessage
from swarmauri.standard.messages.concrete.HumanMessage import HumanMessage
from swarmauri.standard.messages.concrete.SystemMessage import SystemMessage

### **Unit Test: `test_ubc_resource`**

This unit test verifies that the `MistralModel`  is initialized correctly and that its `resource` attribute is set to `'LLM'`. This is important to ensure that the model is correctly identified when used within your applications.

In [12]:
@pytest.mark.unit
def test_ubc_resource():
    API_KEY = os.getenv('MISTRAL_API_KEY')
    llm = LLM(api_key = API_KEY)
    assert llm.resource == 'LLM'




### **Unit Test: `test_ubc_type`**

This unit test ensures that the `MistralModel`  is correctly initialized and that its `type` attribute is set to `'MistralModel'`. This check is essential to verify that the model is recognized by its specific class name within the SDK.

In [3]:
@pytest.mark.unit
def test_ubc_type():
    API_KEY = os.getenv('MISTRAL_API_KEY')
    llm = LLM(api_key = API_KEY)
    assert llm.type == 'MistralModel'

### **Unit Test: `test_serialization`**

This unit test checks the serialization and deserialization process for the `MistralModel` . It ensures that the model's state can be accurately converted to a JSON format and then restored back to an object without losing any data. This is crucial for scenarios where the model's state needs to be saved and loaded, such as in persistent storage or data transfer.

In [4]:
@pytest.mark.unit
def test_serialization():
    API_KEY = os.getenv('MISTRAL_API_KEY')
    llm = LLM(api_key = API_KEY)
    assert llm.id == LLM.model_validate_json(llm.model_dump_json()).id

### **Unit Test: `test_default_name`**

This unit test verifies that the `MistralModel`  is initialized with the correct default name. Ensuring that the model has the correct default name is important for consistency and for identifying the model version being used.


In [5]:
@pytest.mark.unit
def test_default_name():
    API_KEY = os.getenv('MISTRAL_API_KEY')
    model = LLM(api_key = API_KEY)
    assert model.name == 'open-mixtral-8x7b'

### Unit Test: `test_no_system_context`

This unit test ensures that the `MistralModel`  can generate a prediction in a conversation without any predefined system context. It checks that the model's response is of the correct type, verifying basic functionality in a conversational setup.


In [6]:
@pytest.mark.unit
def test_no_system_context():
    API_KEY = os.getenv('MISTRAL_API_KEY')
    model = LLM(api_key = API_KEY)
    conversation = Conversation()

    input_data = "Hello"
    human_message = HumanMessage(content=input_data)
    conversation.add_message(human_message)

    model.predict(conversation=conversation)
    prediction = conversation.get_last().content
    assert type(prediction) == str

### **Acceptance Test: `test_nonpreamble_system_context`**

This acceptance test checks how the `MistralModel`  responds when a system context is introduced mid-conversation. Specifically, it verifies that the model adheres to the system instruction provided and incorporates it into its subsequent responses.


In [8]:
@pytest.mark.acceptance
def test_nonpreamble_system_context():
    API_KEY = os.getenv('MISTRAL_API_KEY')
    model = LLM(api_key = API_KEY)
    conversation = Conversation()

    # Say hi
    input_data = "Hi"
    human_message = HumanMessage(content=input_data)
    conversation.add_message(human_message)

    # Get Prediction
    prediction = model.predict(conversation=conversation)

    # Give System Context
    system_context = 'You only respond with the following phrase, "Jeff"'
    human_message = SystemMessage(content=system_context)
    conversation.add_message(human_message)

    # Prompt
    input_data = "Hello Again."
    human_message = HumanMessage(content=input_data)
    conversation.add_message(human_message)


    model.predict(conversation=conversation)
    prediction = conversation.get_last().content
    assert 'Jeff' in prediction

### **Unit Test: `test_preamble_system_context`**

This unit test checks how the `MistralModel`  handles a system context when it is provided at the start of a conversation. It ensures that the model follows the given instruction and includes the expected response phrase in its output.


In [9]:
@pytest.mark.unit
def test_preamble_system_context():
    API_KEY = os.getenv('MISTRAL_API_KEY')
    model = LLM(api_key = API_KEY)
    conversation = Conversation()

    system_context = 'You only respond with the following phrase, "Jeff"'
    human_message = SystemMessage(content=system_context)
    conversation.add_message(human_message)

    input_data = "Hi"
    human_message = HumanMessage(content=input_data)
    conversation.add_message(human_message)

    model.predict(conversation=conversation)
    prediction = conversation.get_last().content
    assert type(prediction) == str
    assert 'Jeff' in prediction


### **Acceptance Test: `test_multiple_system_contexts`**

This acceptance test verifies that the `MistralModel`  correctly handles multiple system contexts introduced at different points in the conversation. It checks that the model updates its response based on the most recent system context and includes the expected phrase in its output.


In [ ]:
@pytest.mark.acceptance
def test_multiple_system_contexts():
    API_KEY = os.getenv('MISTRAL_API_KEY')
    model = LLM(api_key = API_KEY)
    conversation = Conversation()

    system_context = 'You only respond with the following phrase, "Jeff"'
    human_message = SystemMessage(content=system_context)
    conversation.add_message(human_message)

    input_data = "Hi"
    human_message = HumanMessage(content=input_data)
    conversation.add_message(human_message)

    model.predict(conversation=conversation)

    system_context_2 = 'You only respond with the following phrase, "Ben"'
    human_message = SystemMessage(content=system_context_2)
    conversation.add_message(human_message)

    input_data_2 = "Hey"
    human_message = HumanMessage(content=input_data_2)
    conversation.add_message(human_message)

    model.predict(conversation=conversation)
    prediction = conversation.get_last().content
    assert type(prediction) == str
    assert 'Ben' in prediction